In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import time
import sys

In [9]:
cap = cv2.VideoCapture('../pubg/456(수정).m4v')


if cap is None : 
    print('Video read failed')
    sys.exit()
                    
    
while True : 
    ret, frame = cap.read()
    
    if ret == False :
        break
    
    if cv2.waitKey(10) == 27 :
        break
        
    resize_frame = cv2.resize(frame, (1280, 720), interpolation=cv2.INTER_CUBIC)
    cv2.imshow('video', resize_frame)   
    
cap.release()
cv2.destroyAllWindows()

 BackgroundSubtractorMOG2 

In [5]:
## 잘되는데 주석처리 2208300927
## 후보1, 백그라운드 마스크 처리를 통해서 제외된 것을 인식함
# cv2.createBackgroundSubtractorMOG2() 함수 이용

cap = cv2.VideoCapture('../pubg/final test 2.mp4')

# 옵션 설명 http://layer0.authentise.com/segment-background-using-computer-vision.html

fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=100, detectShadows=0)

while(1):
    ret, frame = cap.read()
    width = frame.shape[1]
    height = frame.shape[0]
    frame = cv2.resize(frame, (int(width/3), int(height/3)))
    fgmask = fgbg.apply(frame)

    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(fgmask)


    for index, centroid in enumerate(centroids): # 중심에 대한 순서값 부여
        if stats[index][0] == 0 and stats[index][1] == 0:
            continue
        if np.any(np.isnan(centroid)):
            # isnan 함수는 데이터가 nan(not a number)인지 아닌지를 판별해주는 함수입니다. 
            # nan이면 True를, nan이 아니면 False를 반환
            continue


        x, y, width, height, area = stats[index]
        centerX, centerY = int(centroid[0]), int(centroid[1])

        if area > 5:
            
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 0, 255))


    cv2.imshow('mask',fgmask)
    cv2.imshow('frame',frame)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
## googlenet 으로 진행했던 것 응용, fullbody 인식
## 후보2
## 처리할게 많아서 그런지 너무느림, 구글링 결과 영상이 빨라질려고 하면, tensorflow를 통해서 gpu를 가동해야한다고함.. 
## 여러가지 시도를 해보았으나, 버전이 계속 맞지 않아서 그런지 기동불가

cap = cv2.VideoCapture('../pubg/456(수정).m4v')

body_cascade = cv2.CascadeClassifier('../googlenet/haarcascade_fullbody.xml')

prev_time = 0
FPS = 10

if cap is None : 
    print('Video read failed')
    sys.exit()
                    
while True : 
    ret, frame = cap.read()
    
    grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    body = body_cascade.detectMultiScale(grayframe, 1.01, 10, minSize=(30, 30))
    for (x,y,w,h) in body :         
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3, 4, 0)
    
    #cv2.imshow('body', frame)
    
    if ret == False :
        break
    
    if cv2.waitKey(10) == 27 :
        break
        
    resize_frame = cv2.resize(frame, (960, 640), interpolation=cv2.INTER_CUBIC)
    cv2.imshow('video', resize_frame)   
    
    
cap.release()
cv2.destroyAllWindows()

In [3]:
## 2208260923 test 1
## 후보3 (이게 가장 베스트)
## 객체 드래그 해서 추적함
## https://bkshin.tistory.com/entry/OpenCV-32-객체-추적을-위한-Tracking-API
## https://rosia.tistory.com/243 -> opencv tracker 종류
# 트랙커 객체 생성자 함수 리스트 ---①

trackers = [cv2.TrackerMIL_create,
            cv2.TrackerKCF_create,
            cv2.TrackerGOTURN_create, # opencv tracking API 모듈들
            cv2.TrackerCSRT_create]  # 모듈의 버젼이 다를 수 있으니, 모듈 확인 후 진행할 것 !!

trackerIdx = 0  # 트랙커 생성자 함수 선택 인덱스
tracker = None
isFirst = True

video_src = 0 # 비디오 파일과 카메라 선택 ---②
video_src = "../pubg/final test 1.mp4"

cap = cv2.VideoCapture(video_src)
fps = cap.get(cv2.CAP_PROP_FPS) # 프레임 수 구하기

win_name = 'Tracking APIs'
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print('Cannot read video file')
        break
    img_draw = frame.copy()
    if tracker is None: # 트랙커 생성 안된 경우
        cv2.putText(img_draw, "spacebar start or stop", \
            (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2,cv2.LINE_AA)
    else:
        ok, bbox = tracker.update(frame)   # 새로운 프레임에서 추적 위치 찾기 ---③
        (x,y,w,h) = bbox
        if ok: # 추적 성공
            cv2.rectangle(img_draw, (int(x), int(y)), (int(x + w), int(y + h)), \
                          (0,255,0), 2, 1)
        else : # 추적 실패시
            cv2.putText(img_draw, "tracking failed.", (100,80), \
                        cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2,cv2.LINE_AA)
            
    trackerName = tracker.__class__.__name__


    cv2.imshow(win_name, img_draw)
    
    key = cv2.waitKey(50) & 0xff
    # 스페이스 바 또는 비디오 파일 최초 실행 ---④
    if key == ord(' ') or (video_src != 0 and isFirst): 
        isFirst = False
        roi = cv2.selectROI(win_name, frame, False)  # 초기 객체 위치 설정
        if roi[2] and roi[3]:         # 위치 설정 값 있는 경우
            tracker = trackers[trackerIdx]()    #트랙커 객체 생성 ---⑤
            isInit = tracker.init(frame, roi)
    elif key in range(48, 56): # 0~7 숫자 입력   ---⑥
        trackerIdx = key-48     # 선택한 숫자로 트랙커 인덱스 수정
        if bbox is not None:
            tracker = trackers[trackerIdx]() # 선택한 숫자의 트랙커 객체 생성 ---⑦
            isInit = tracker.init(frame, bbox) # 이전 추적 위치로 추적 위치 초기화
    elif key == 27 : 
        break
else:
    print( "read is failed vedio")
    
cap.release()
cv2.destroyAllWindows()

Cannot read video file


In [3]:
## https://deep-learning-study.tistory.com/275
# 후보4
## 실행은 되긴하나 HSV에 관한게 필요할 듯, 해당 알고리즘에 대한 이해 필요


cap = cv2.VideoCapture('../pubg/final test 3.mp4') # 비디오 파일 열기

if not cap.isOpened():
    print('Video open failed!')
    sys.exit()

x, y, w, h = 135, 220, 100, 100 # 초기 사각형 영역 설정: (x, y, w, h)
rc = (x, y, w, h)

ret, frame = cap.read() # 영상의 정보 받아오기

if not ret:
    print('frame read failed!')
    sys.exit()
    
roi = frame[y:y+h, x:x+w]
roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)


channels = [0,1] # # HS 히스토그램 계산, H와 S만 이용. V는 안씀
ranges = [0, 180, 0, 256]
# calcHist(images, channels, mask, histSize, ranges[, hist[, accumulate]]) -> hist
hist = cv2.calcHist([roi_hsv], channels, None, [90, 128], ranges)

# Mean Shift 알고리즘 종료 기준
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

# 비디오 매 프레임 처리
while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    # HS 히스토그램에 대한 역투영
    # frame을 HSV로 변환
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # 히스토그램 역투영 확률 데이터 얻기
    backproj = cv2.calcBackProject([frame_hsv], channels, hist, ranges, 1)
    
    # Mean Shift
    # 역투영 확률값을 Mean shift 인자에 입력
    _, rc = cv2.meanShift(backproj, rc, term_crit)
    
    # 추적 결과 화면 출력
    cv2.rectangle(frame, rc, (0, 0, 255), 2)
    resize_frame = cv2.resize(frame, (1280, 720), interpolation=cv2.INTER_CUBIC)
    cv2.imshow('frame', resize_frame)
    
    if cv2.waitKey(60) == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

In [4]:
## 2208261529 되긴함 근데 프레임 오지게 끊김
# 후보5
# yolo 모델을 이용한 객체 추적, 진행하였으나, 후보 2처럼 tensorflow의 gps 를 연동하지 않으면 프레임 끊김은 지속될 것으로 확인
# 나중에 좀 더 자세히 알 필요가 있음

vedio_path = '../pubg/final test 3.mp4' #-- 사용할 영상 경로
min_confidence = 0.5

def detectAndDisplay(frame):
    start_time = time.time()
    img = cv2.resize(frame, None, fx=0.8, fy=0.8)
    height, width, channels = img.shape

    #-- 창 크기 설정
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    #-- 탐지한 객체의 클래스 예측 
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > min_confidence:
                # 탐지한 객체 박싱
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
               
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)
    font = cv2.FONT_HERSHEY_DUPLEX
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = "{}: {:.2f}".format(classes[class_ids[i]], confidences[i]*100)
           # print(i, label)
            color = colors[i] #-- 경계 상자 컬러 설정 / 단일 생상 사용시 (255,255,255)사용(B,G,R)
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y - 5), font, 1, color, 1) 
    end_time = time.time()
    process_time = end_time - start_time
    #  print("=== A frame took {:.3f} seconds".format(process_time))
    cv2.imshow("YOLO test", img)
    
#-- yolo 포맷 및 클래스명 불러오기
net = cv2.dnn.readNet('../darknet-master/yolov3.cfg','../darknet-master/yolov3.weights')

#-- GPU 사용
#net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
#net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

#-- 클래스(names파일) 오픈 / 본인 개발 환경에 맞게 변경할 것
classes = []
with open("../darknet-master/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

#-- 비디오 활성화
cap = cv2.VideoCapture(vedio_path) #-- 웹캠 사용시 vedio_path를 0 으로 변경
if not cap.isOpened:
    # print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        # print('--(!) No captured frame -- Break!')
        break
    detectAndDisplay(frame)
    #-- q 입력시 종료
    if cv2.waitKey(50) & 0xFF == 27 :
        break

cv2.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: ../darknet-master/yolov3.cfg in function 'cv::dnn::dnn4_v20220524::readNetFromDarknet'
